<a href="https://colab.research.google.com/github/graviraja/100-Days-of-NLP/blob/applications%2Fgeneration/applications/generation/Machine%20Translation%20with%20Bahdanau%20Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Translation with Bahdanau Attention

Here in this notebook, we will be adding attention to our translation model.

Go through the following resources:

- [Basic Machine Translation](https://github.com/graviraja/100-Days-of-NLP/blob/master/applications/generation/Basic%20Machine%20Translation.ipynb)

- [Improved Machine Translation](https://github.com/graviraja/100-Days-of-NLP/blob/master/applications/generation/Improved%20Machine%20Translation.ipynb)

- [Bahdanau Attention](https://github.com/graviraja/100-Days-of-NLP/blob/master/architectures/bahdanau_attention.py)

![arch](https://drive.google.com/uc?id=1qre87BEtVB8QGEiI08ooE6DBQv8Yf1wA)


In [0]:
import math
import time
import random
import spacy
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchtext
from torchtext.data import Field, BucketIterator
from torchtext.datasets import Multi30k

In [0]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [3]:
!python -m spacy download de
!python -m spacy download en

     |████████████████████████████████| 14.9MB 1.7MB/s 
  Created wheel for de-core-news-sm: filename=de_core_news_sm-2.2.5-cp36-none-any.whl size=14907056 sha256=bcb29cbcf5f9c2e7a84338387b23ecc67038aae76e030436068677382466ecc0
  Stored in directory: /tmp/pip-ephem-wheel-cache-cll2bt6x/wheels/ba/3f/ed/d4aa8e45e7191b7f32db4bfad565e7da1edbf05c916ca7a1ca
Successfully built de-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/de
You can now load the model via spacy.load('de')
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [0]:
spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

In [0]:
def tokenize_de(text):
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [0]:
SRC = Field(
    tokenize = tokenize_de,
    init_token = '<sos>',
    eos_token = '<eos>',
    lower = True
)

In [0]:
TRG = Field(
    tokenize = tokenize_en,
    init_token = '<sos>',
    eos_token = '<eos>',
    lower = True
)

In [0]:
train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'), fields = (SRC, TRG))

In [0]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [0]:
BATCH_SIZE = 64

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    device = device
)

In [0]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, pad_token):
        super().__init__()

        self.embedding = nn.Embedding(input_dim, emb_dim, padding_idx=pad_token)
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional=True)
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, src):
        # src => [src_len, batch_size]

        embedded = self.embedding(src)
        embedded = self.dropout(embedded)
        # embedded => [src_len, batch_size, emb_dim]

        output, hidden = self.rnn(embedded)
        # output => [src_len, batch_size, hidden_dim * 2]
        # hidden => [2, batch_size, hidden_dim]

        final_forward_hidden = hidden[0, :, :]
        final_backward_hidden = hidden[1, :, :]

        combined_hidden = torch.cat((final_forward_hidden, final_backward_hidden), dim=1)
        # combined_hidden => [batch_size, hidden_dim * 2]

        decoder_initial_hidden = self.fc(combined_hidden).tanh()
        # decoder_initial_hidden => [batch_size, hidden_dim]

        return output, decoder_initial_hidden

In [0]:
class BahdanauAttention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()

        self.w1 = nn.Linear(dec_hid_dim, dec_hid_dim)
        self.w2 = nn.Linear(enc_hid_dim * 2, dec_hid_dim)

        self.v = nn.Parameter(torch.rand(dec_hid_dim * 2))
        stdv = 1. / math.sqrt(dec_hid_dim * 2)
        self.v.data.normal_(mean=0, std=stdv)

    def forward(self, hidden, encoder_outputs):
        # hidden => [batch_size, hidden_dim]
        # encoder_outputs => [src_len, batch_size, hidden_dim * 2]
        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]

        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        # hidden => [batch_size, src_len, hidden_dim]

        encoder_outputs = encoder_outputs.permute(1, 0 , 2)
        # encoder_outputs => [batch_size, src_len, hidden_dim * 2]

        hidden_energy = self.w1(hidden)
        # hidden_energy => [batch_size, src_len, hidden_dim]

        encoder_output_energy = self.w2(encoder_outputs)
        # encoder_output_energy => [batch_size, src_len, hidden]

        combined_energy = torch.cat((hidden_energy, encoder_output_energy), dim=2)
        # combined_energy => [batch_size, src_len, hidden_dim * 2]

        energy = self.v * combined_energy
        # energy => [batch_size, src_len, hidden_dim * 2]

        energy = torch.sum(energy, dim=2)
        # energy => [batch_size, src_len]

        attention_energy = F.softmax(energy, dim=1)
        # attention_energy => [batch_size, src_len]

        return attention_energy


In [0]:
class Decoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hidden_dim, attn, dropout, pad_token):
        super().__init__()

        self.output_dim = input_dim
        self.embedding = nn.Embedding(input_dim, emb_dim, padding_idx=pad_token)
        self.rnn = nn.GRU(emb_dim + hidden_dim * 2, hidden_dim)
        self.attn = attn
        self.fc = nn.Linear(emb_dim + hidden_dim * 3, input_dim)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, trg, hidden, encoder_outputs):
        # trg => [batch_size]
        # hidden => [batch_size, hidden_dim]
        # encoder_outputs => [src_len, batch_size, hidden_dim * 2]

        input = trg.unsqueeze(0)
        # input => [1, batch_size]
        
        embedded = self.embedding(input)
        embedded = self.dropout(embedded)
        # embedded => [1, batch_size, emb_dim]

        attention_energies = self.attn(hidden, encoder_outputs)
        # attention_energies => [batch_size, src_len]

        attention_energies = attention_energies.unsqueeze(1)
        # attention_energies => [batch_size, 1, src_len]

        encoder_outputs = encoder_outputs.permute(1, 0 , 2)
        # encoder_outputs => [batch_size, src_len, hidden_dim * 2]

        weighted = torch.bmm(attention_energies, encoder_outputs)
        # weighted => [batch_size, 1, hidden_dim * 2]

        weighted = weighted.permute(1, 0, 2)
        # weighted => [1, batch_size, hidden_dim * 2]

        combined_rnn_input = torch.cat((embedded, weighted), dim=2)
        # combined_input => [1, batch_size, hidden_dim * 2 + emb_dim]

        output, hidden = self.rnn(combined_rnn_input, hidden.unsqueeze(0))
        # output => [1, batch_size, hidden_dim]
        # hidden => [1, batch_size, hidden_dim]

        combined_out_input = torch.cat((output, weighted, embedded), dim=2)
        # combined_out_input => [1, batch_size, hidden_dim * 3 + emb_dim]

        combined_out_input = combined_out_input.squeeze(0)
        # combined_out_input => [batch_size, hidden_dim * 3 + emb_dim]

        logits = self.fc(combined_out_input)
        # logits => [batch_size, output_dim]
        
        return logits, hidden.squeeze(0)

        

In [0]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        # src => [src_len, batch_size]
        # trg => [trg_len, batch_size]

        encoder_outputs, hidden = self.encoder(src)
        # encoder_outputs => [src_len, batch_size, hidden_dim * 2]
        # hidden => [batch_size, hidden_dim]

        trg_len = trg.shape[0]
        batch_size = trg.shape[1]
        output_dim = self.decoder.output_dim

        outputs = torch.zeros(trg_len, batch_size, output_dim).to(self.device)
        dec_inp = trg[0, :]

        for i in range(1, trg_len):
            output, hidden = self.decoder(dec_inp, hidden, encoder_outputs)
            # output => [batch_size, output_dim]
            # hidden => [batch_size, hidden_dim]
            outputs[i] = output

            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.argmax(1)
            dec_inp = trg[i] if teacher_force else top1
        return outputs
        

In [0]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

SRC_PAD_IDX = SRC.vocab.stoi[SRC.pad_token]
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

attn = BahdanauAttention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT, SRC_PAD_IDX)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, DEC_HID_DIM, attn, DEC_DROPOUT, TRG_PAD_IDX)

model = Seq2Seq(enc, dec, device).to(device)

In [108]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7855, 256, padding_idx=1)
    (rnn): GRU(256, 512, bidirectional=True)
    (fc): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(5893, 256, padding_idx=1)
    (rnn): GRU(1280, 512)
    (attn): BahdanauAttention(
      (w1): Linear(in_features=512, out_features=512, bias=True)
      (w2): Linear(in_features=1024, out_features=512, bias=True)
    )
    (fc): Linear(in_features=1792, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [109]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 20,519,941 trainable parameters


In [0]:
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [0]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()

        epoch_loss += loss.item()
    
    return epoch_loss / len(iterator)

In [0]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)


In [0]:

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [114]:

N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 3m 21s
	Train Loss: 4.675 | Train PPL: 107.210
	 Val. Loss: 4.517 |  Val. PPL:  91.597
Epoch: 02 | Time: 3m 19s
	Train Loss: 3.659 | Train PPL:  38.819
	 Val. Loss: 3.941 |  Val. PPL:  51.444
Epoch: 03 | Time: 3m 21s
	Train Loss: 3.087 | Train PPL:  21.920
	 Val. Loss: 3.569 |  Val. PPL:  35.485
Epoch: 04 | Time: 3m 20s
	Train Loss: 2.697 | Train PPL:  14.841
	 Val. Loss: 3.491 |  Val. PPL:  32.812
Epoch: 05 | Time: 3m 20s
	Train Loss: 2.349 | Train PPL:  10.477
	 Val. Loss: 3.458 |  Val. PPL:  31.758
Epoch: 06 | Time: 3m 20s
	Train Loss: 2.081 | Train PPL:   8.015
	 Val. Loss: 3.432 |  Val. PPL:  30.954
Epoch: 07 | Time: 3m 20s
	Train Loss: 1.868 | Train PPL:   6.475
	 Val. Loss: 3.464 |  Val. PPL:  31.943
Epoch: 08 | Time: 3m 20s
	Train Loss: 1.721 | Train PPL:   5.591
	 Val. Loss: 3.509 |  Val. PPL:  33.411
Epoch: 09 | Time: 3m 19s
	Train Loss: 1.582 | Train PPL:   4.865
	 Val. Loss: 3.650 |  Val. PPL:  38.458
Epoch: 10 | Time: 3m 20s
	Train Loss: 1.455 | Train PPL

In [115]:
model.load_state_dict(torch.load('model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 3.395 | Test PPL:  29.825 |
